# Algoritimo Simplex utilizando Python and Numpy. #

Simplex é um método Iterativo para solução de problemas de Otimização Linear, com um número finito de passos leva à solução ótima do problema.

## Algoritmo do Método Simplex ##

### Para a resolução de um problema usando simplex são utilizados os seguintes passos: ###
1. Adicionar variáveis de folga para cada uma das desigualdades;
2. Montar uma tabela para os cálculos, colocando todos os coeficientes das restrições e da função objetivo modificada;
3. Determinar uma solução básica factível inicial e/ou verificar se a solução é ótima, se todos os valores da linha correspondente à função objetivo são positivos ou nulos, então a solução atual é ótima;
4. Verificar qual variável entra na base, uma candidata a entrar na base é a variável que apresenta o valor mais negativo na linha da função objetivo. Isto é, apresenta maior contribuição para o aumento da do valor da função objetivo. Esta será a coluna pivô;
5. Determinar o elemento que sai da base, dividir os
valores da coluna b, pelos valores das respectivas
variáveis na coluna pivô. A variável referente ao menor
valor da divisão será a que sairá da base.
6. Recalcular a tabela fazendo com que a coluna
referente à variável que entra na base se torne uma coluna
identidade, idêntica à coluna da variável que sai da base.
7. Retornar ao Passo **3**.

## Simplex - AC1 ##

Agora introduzido no assunto é o momento de entender o propózito deste código.
Temos como objetivo escrever um código que calcule o método **Simplex** para problemas de **maximização**, que tem como objetivo a avaliação dos integrantes envolvidos. 

**Linguagem de programação utilizada:** Python

**Entregáveis:**
- código-fonte;
- notebook com código comentado.

**Arquivo de entrada:**

- deve respeitar o formato padrão desse [exemplo de entrada](./exemplo_01_entrada.txt), sabendo que TODAS as restrições possuem sinal de menor ou igual (<=);
- considerar que o arquivo de entrada estará sempre na mesma pasta que o código.

## Código explicado ##

1. Será necessário um metodo de leitura de arquivo para a entrada dos dados:
> Considerando que o arquivo que será lido esá na pasta raiz e que existe apenas um arquivo `.txt` 

In [5]:
import os

def get_values():
    lines = []
    files = os.listdir('.')
    for f in files:
        if f.endswith(".txt"):
            f = open(f)
            for line in f:
                lines.append(line.replace("\n", "").split(" "))
    variables = len(lines[0])
    restrictions = len(lines) - 1
    return lines, variables, restrictions

2. Com os valores obtidos com o arquivo será gerado uma matriz numpy com linhas suficientes para cada restrição mais a função objetivo e colunas suficientes para as variáveis, variáveis de folga, M (máx.) E o valor correspondente.

In [7]:
import numpy as np 

def generate_matrix(variables, restrictions):    
    matrix = np.zeros((restrictions + 1, variables + restrictions + 2))    
    return matrix

3. Agora que a tabela está montada é hora de hora de separar a função objetivo das restrições. 

In [9]:
def get_objetive_restrictions(lines):
    objetive = []
    restrictions = []
    for line in lines:
        if lines.index(line) == 0:
            objetive = line
        else:
            restrictions.append(line)
    return objetive, restrictions

4. Os valores obtidos no arquivo deveram ser convertidos de string em variáveis ​​float

In [11]:
def convert(equation):
    equation = [float(value) for value in equation]
    return equation

5. Com esses valores do arquivo e com o metodo escrito anteriormente, agora é o momento de preencher as restrições do problema na tabela. 

In [13]:
def constrain(table,restrictions):
    for restriction in restrictions:    
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1
        j = 0
        while j < lr:
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:
                row = row_check
                break
            j +=1 
        restriction = convert(restriction)
        i = 0
        while i<len(restriction)-1:
            row[i] = restriction[i]
            i +=1
        row[-1] = restriction[-1]
        row[var+j] = 1

6. Com a tabela já preenchida com as restrições é hora de adicionar a função objetivo à tabela.

In [15]:
def obj(table,eq):
    eq = convert(eq)
    lr = len(table[:,0])
    row = table[lr-1,:]
    for i in range(len(eq)):
        row[i] = eq[i]*-1
    row[-2] = 1

A seguir, vamos verificar se 1+ pivôs são necessários devido a um elemento negativo na coluna mais à direita, excluindo o valor inferior, é claro.

In [17]:
def next_round_r(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True

Da mesma forma, verificaremos se mais de 1 pivôs são necessários devido a um elemento negativo na linha inferior, excluindo o valor final.

In [19]:
def next_round(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True

E agora que criamos funções que retornam booleanos, sejam ou não necessários pivôs adicionais, precisamos determinar onde esses elementos estão localizados. Começaremos encontrando elementos negativos na coluna mais à direita.

In [21]:
def find_neg_r(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

Da mesma forma, precisamos localizar os elementos negativos na linha inferior

In [23]:
def find_neg(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

Recapitulando rapidamente, identificamos os índices de coluna e linha, respectivamente, para elementos negativos na última coluna, última linha. Mas precisamos dar um passo adiante e encontrar o elemento pivô correspondente a esses valores.

In [25]:
def loc_piv_r(table):
    total = []        
    r = find_neg_r(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]

Ok, isso é um pouco terrível de se olhar, então vamos dissecá-lo. Se você se lembra do vídeo, se vemos um elemento negativo na coluna mais à direita, precisamos procurar o valor mais negativo na mesma linha do quadro. Depois de encontrado, compararemos a coluna em que esse elemento está com a coluna final. Procuramos a relação menos positiva. A variável i itera na coluna que contém o elemento negativo, enquanto a variável b itera na coluna final. Se i NÃO for zero e b / i for maior que zero, acrescentamos à nossa lista total. Se ambas as condições não forem satisfeitas, inserimos 10.000 como espaço reservado. Em teoria, se estivéssemos otimizando números muito, muito grandes, essa não seria uma solução eficaz. Mas com números de tamanho razoável, 10.000 nunca será selecionado como a "proporção menos positiva".
A seguir, vamos realizar uma tarefa muito semelhante; precisamos encontrar um elemento pivô correspondente a um elemento negativo na linha inferior.

In [27]:
def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if b/i >0 and i**2>0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]

Agora, vamos finalmente construir a função pivô! O que precisamos fazer é girar sobre um elemento para remover a entrada negativa na coluna ou linha final e retornar a tabela atualizada.

In [29]:
def pivot(row,col,table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Não é possível girar neste elemento.')

Até agora, todas as funções que desenvolvemos são úteis para resolver problemas de maximização e minimização. O seguinte é necessário para resolver os problemas de minimização. Em problemas de maximização, M é o oposto da função objetivo. Ex: 4 (x1) + 2 (x2) torna-se -4 (x1) -2 (x2). No entanto, com problemas de minimização, a função objetivo é deixada "como está", geralmente falando, pois é o oposto da função objetivo de maximização.

In [31]:
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

Você pode notar que nossa função foi projetada para resolver qualquer combinação de variáveis ​​e restrições. Portanto, precisamos construir uma função que irá gerar apenas o número necessário de variáveis ​​x1, x2,… xn. Isso (abaixo) resolverá o problema!

In [32]:
def gen_var(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

Agora, agradeça a si mesmo por sua incrível paciência - finalmente é hora de colocar todos os blocos de construção juntos e criar as funções de maximização e minimização! Essas funções parecerão muito semelhantes, ambas usarão loops while para determinar se 1+ pivô é necessário, localize o elemento pivô, gire sobre ele e continue o processo até que todos os elementos negativos tenham sido removidos da última coluna e linha. Em seguida, as variáveis serão geradas para x1 a xn e valores atribuídos de acordo com suas posições no quadro. Além disso, max receberá seu valor apropriado. Por último, a função retornará o máximo e as variáveis na forma de dicionário.

In [33]:
def maxz(table):
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1,-1]
    return val

In [34]:
if __name__ == "__main__":
    #TODO: Inserir metodo de leitura de arquivo (OK)
    #TODO: Quebrar valores lidos no arquivo em variavies (OK)
    #TODO: Modificar metodo gen_matrix, para receber variaveis (OK)
    lines, variables, restrictions = get_values()
    m = generate_matrix(variables, restrictions)
    objetive, restrictions = get_objetive_restrictions(lines)
    constrain(m,restrictions)
    obj(m,objetive)
    print (m)
    print(maxz(m))

[[ 1.  0.  1.  0.  0.  0.  4.]
 [ 0.  1.  0.  1.  0.  0. 12.]
 [ 3.  2.  0.  0.  1.  0. 18.]
 [-3. -5.  0.  0.  0.  1.  0.]]
{'x1': 0, 'x2': 9.0, 'max': 45.0}
